In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from mediapipe import solutions

def load_trained_model(model_path='sign_language_model.h5'):
    return load_model(model_path)

def extract_keypoints_from_frame(frame, hands):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    keypoints = []
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                keypoints.append([landmark.x, landmark.y, landmark.z])  # Only (x, y, z)
    
    if keypoints:
        return np.array(keypoints).flatten()
    return np.zeros(63)  # 21 keypoints * 3 values (x, y, z)

def preprocess_keypoints(keypoints):
    if len(keypoints) < 63:
        keypoints = np.pad(keypoints, (0, 63 - len(keypoints)), 'constant')
    else:
        keypoints = keypoints[:63]  # Ensure exactly 63 values
    return keypoints

def predict_sign_language(sequence, model, label_map):
    sequence = np.array(sequence).reshape((1, 30, 63))  # Ensure correct shape
    prediction = model.predict(sequence)
    predicted_label_idx = np.argmax(prediction)
    label_name = [label for label, idx in label_map.items() if idx == predicted_label_idx][0]
    return label_name

def process_video(model, label_map, camera_index=0):
    hands = solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    cap = cv2.VideoCapture(camera_index)
    sequence = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        keypoints = extract_keypoints_from_frame(frame, hands)
        processed_keypoints = preprocess_keypoints(keypoints)
        
        sequence.append(processed_keypoints)
        if len(sequence) > 30:
            sequence = sequence[-30:]

        if len(sequence) == 30:
            predicted_label = predict_sign_language(sequence, model, label_map)
            if predicted_label is not None:
                cv2.putText(frame, predicted_label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        cv2.imshow('Sign Language Recognition (Real-time)', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

model_path = 'sign_language_model.h5'
label_map = {
    'A LOT': 0, 'ABUSE': 1, 'AFRAID': 2, 'AGREE': 3, 'ALL': 4, 'ANGRY': 5, 'ANYTHING': 6, 'APPRECIATE': 7, 'BAD': 8, 
    'BEAUTIFUL': 9, 'BECOME': 10, 'BED': 11, 'BORED': 12, 'BRING': 13, 'CHAT': 14, 'CLASS': 15, 'COLD': 16, 
    'COLLEGE_SCHOOL': 17, 'COMB': 18, 'COME': 19, 'CONGRATULATIONS': 20, 'CRYING': 21, 'DARE': 22, 'DIFFERENCE': 23, 
    'DILEMMA': 24, 'DISAPPOINTED': 25, 'DO': 26, "DON'T CARE": 27, 'ENJOY': 28, 'FAVOUR': 29, 'FEVER': 30, 
    'FINE': 31, 'FOOD': 32, 'FREE': 33, 'FRIEND': 34, 'FROM': 35, 'GO': 36, 'GOOD': 37, 'GRATEFUL': 38, 'HAD': 39, 
    'HAPPENED': 40, 'HAPPY': 41, 'HEAR': 42, 'HEART': 43, 'HELLO_HI': 44, 'HELP': 45, 'HIDING': 46, 'HOW': 47, 
    'HUNGRY': 48, 'HURT': 49, 'I_ME_MINE_MY': 50, 'KIND': 51, 'LEAVE': 52, 'LIKE': 53, 'LIKE_LOVE': 54, 'MEAN_IT': 55, 
    'MEDICINE': 56, 'MEET': 57, 'NAME': 58, 'NICE': 59, 'NOT': 60, 'NUMBER': 61, 'OLD_AGE': 62, 'ON THE WAY': 63, 
    'OUTSIDE': 64, 'PHONE': 65, 'PLACE': 66, 'PLEASE': 67, 'POUR': 68, 'PREPARE': 69, 'PROMISE': 70, 'REALLY': 71, 
    'REPEAT': 72, 'ROOM': 73, 'SERVE': 74, 'SHIRT': 75, 'SITTING': 76, 'SLEEP': 77, 'SLOWER': 78, 'SO MUCH': 79, 
    'SOFTLY': 80, 'SOME HOW': 81, 'SOME ONE': 82, 'SOMETHING': 83, 'SORRY': 84, 'SPEAK': 85, 'STOP': 86, 
    'STUBBORN': 87, 'SURE': 88, 'TAKE CARE': 89, 'TAKE TIME': 90, 'TALK': 91, 'TELL': 92, 'THANK': 93, 'THAT': 94, 
    'THINGS': 95, 'THINK': 96, 'THIRSTY': 97, 'TIRED': 98, 'TODAY': 99, 'TRAIN': 100, 'TRUST': 101, 'TRUTH': 102, 
    'TURN ON': 103, 'UNDERSTAND': 104, 'WANT': 105, 'WATER': 106, 'WEAR': 107, 'WELCOME': 108, 'WHAT': 109, 
    'WHERE': 110, 'WHO': 111, 'WORRY': 112, 'YOU': 113
}

model = load_trained_model(model_path)
process_video(model, label_map)

1/1 [==============================] - 0s 32ms/step
